In [2]:
#importing CSV File and Reading it
import pandas as pd
import csv
import pyspark
from pyspark import SparkContext, SparkConf
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.plotly as py
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr, zscore

In [3]:
#Starting the Spark Session
conf=SparkConf().setAppName("CSE545 Project").set("spark.driver.memory", "12g").set("spark.executor.memory", "6g").set("spark.driver.maxResultSize", "6g")
sc=SparkContext(conf=conf)

In [4]:
#Loading GHI Dataset
GHI_rdd1 = sc.textFile("002_AppendixD.tsv").map(lambda line: line.split("\t"))

In [5]:
#Deleting records with more than 1 missing values
def count_x(x):
    cnt = 0
    for i in range(0,len(x)):
        if x[i] == " -":
            cnt += 1
    if cnt > 1:
        return False
    return True

In [6]:
GHI_rdd1 = GHI_rdd1.filter(lambda line: len(line)>1).map(lambda x: [w.replace("\"","") for w in x]).filter(lambda x: count_x(x))

In [7]:
#Converting Strings to floats wherever necessary
def conv_x(x):
    if x[0] != "Country":
        x[1] = float(x[1])
        x[2] = float(x[2])
        x[3] = float(x[3])
        x[4] = float(x[4])
    
    return x

In [8]:
#Transforming <5 values into integer values
def rep_5(x):
    a = []
    cnt = 0
    for i in range(0,len(x)):
        if x[i] == "<5":
            x[i] = 4 - cnt
            x[i] = str(x[i])
            cnt += 1
        a.append(x[i])
        
    return a

In [9]:
#Filling in Missing Values based on the future differences
def fill_x(x):
    if x[1] == " -":
        s = float(x[2]) - float(x[3])
        d = float(x[3]) - float(x[4])
        j = (s + d)/2
        x[1] = str(round(j + float(x[2]),1))
        
    return x

In [10]:
GHI_rdd2 = GHI_rdd1.map(lambda x: rep_5(x)).map(lambda x: fill_x(x)).map(lambda x: conv_x(x))
#print(GHI_rdd2.collect())
#Countries whose records are there in the current dataset
GHI_rdd2_unique_val = GHI_rdd2.map(lambda x: x[0]).distinct().collect()

In [11]:
#Loading other values dataset
GHI_rdd3 = sc.textFile("001_AppendixC.tsv").map(lambda line: line.split("\t")).filter(lambda line: len(line)>1).map(lambda x: [w.replace("\"","") for w in x])

In [12]:
#Filtering values which have corresponding record in GHI dataset
GHI_rdd3 = GHI_rdd3.filter(lambda x: x[0] in GHI_rdd2_unique_val)

In [13]:
#Formatting dataset
GHI_rdd3 = GHI_rdd3.map(lambda x: [x[0],[x[1],x[5],x[9],x[13]],[x[2],x[6],x[10],x[14]],[x[3],x[7],x[11],x[15]],[x[4],x[8],x[12],x[16]]])

In [14]:
#CHeking for the number ofmissing values
def cnt(x):
    a = []
    for i in range(0,len(x)):
        c = 0
        for j in range(0,len(x[i])):
            if x[i][j] == "-":
                c += 1
        a.append(c)
    return a

In [15]:
#Removing the records with at least 2 missing values
def mor_2(x):
    for i in range(0,len(x[1])):
        if x[1][i] == 2:
            return x[0]

In [16]:
GHI_rdd3_toRemove = GHI_rdd3.map(lambda x: (x[0],cnt(x[1:5]))).map(lambda x: mor_2(x)).filter(lambda x: x!=None).collect()

In [17]:
#Updating GHI and values datasets for the final filtered results
GHI_rdd4 = GHI_rdd3.filter(lambda x: x[0] not in GHI_rdd3_toRemove)
GHI_rdd2 = GHI_rdd2.filter(lambda x: x[0] not in GHI_rdd3_toRemove)

In [18]:
#Filling in missing values according to the GHI value
def cal_x0(x,ghi_rdd):
    if x[0] != "Country":
        for j in range(0,len(ghi_rdd.value)):
            if x[0] == ghi_rdd.value[j][0]:
                ghis = ghi_rdd.value[j]
        for i in range(1,len(x)):
            ghi = ghis[i]
            if x[i][0] == "-":
                x[i][1] = round(float(x[i][1]),1)
                x[i][2] = round(float(x[i][2]),1)
                x[i][3] = round(float(x[i][3]),1)
                x[i][0] = round(3*(ghi - (x[i][3]/3) - (x[i][2]/6) - (x[i][1]/6)),1)
            else:
                x[i][0] = round(float(x[i][0]),1)
                x[i][1] = round(float(x[i][1]),1)
                x[i][2] = round(float(x[i][2]),1)
                x[i][3] = round(float(x[i][3]),1)
    return x

In [19]:
def conv_listx(x):
    a = [x[0]]
    for i in range(1,5):
        for j in range(0,4):
            a.append(x[i][j])
    return a

In [20]:
#Call to fill in the missing values
GHI_rdd2_broadcast=sc.broadcast(GHI_rdd2.collect())
GHI_rdd4 = GHI_rdd4.map(lambda x: cal_x0(x,GHI_rdd2_broadcast)).map(lambda x: conv_listx(x))

In [21]:
# #Transforming rdd to pandas dataframe for applying ML and Visulaization techniques
# headers = GHI_rdd2.collect()[0]
# GHI_rdd2=GHI_rdd2.filter(lambda x: x[0]!='Country')

# df0 = pd.DataFrame(GHI_rdd2.collect(), columns=headers)

# headers = GHI_rdd4.collect()[0]
# GHI_rdd4=GHI_rdd4.filter(lambda x: x[0]!='Country')

# df1 = pd.DataFrame(GHI_rdd4.collect(), columns=headers)

In [53]:
def calc_beta(betas, X_test, y_test):
    y_pred = np.matmul(X_test, betas)[:,0]
    print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))

In [54]:
def RidgeRegression(X, penalty_value = 1.0, learning_rate = 0.00000000001, n_epochs = 100):

    #Dividing into training and test data
    offset = int(int(X.shape[0]) * 0.9)
    X_test, Y_test = X[offset:,:3], X[offset:,3:]
    X_tf, Y_tf = X[:offset,:3], X[:offset,3:]

    # Conversion to tensors
    X_tf = tf.constant(X_tf, dtype=tf.float32, name="GHI_X")
    Y_tf = tf.constant(Y_tf.reshape(-1,1), dtype=tf.float32, name="GHI_Y")
    Xt_tf = tf.transpose(X_tf)
    penalty = tf.constant(1.0, dtype=tf.float32, name="penalty")
    I = tf.constant(np.identity(int(X_tf.shape[1])), dtype=tf.float32, name="I")
    beta = tf.Variable(tf.random_uniform([int(X_tf.shape[1]), 1], -1., 1.), name = "beta")
    y_pred = tf.matmul(X_tf, beta, name="predictions")
    penalizedCost = tf.reduce_sum(tf.square(Y_tf - y_pred)) + penalty * tf.reduce_sum(tf.square(beta))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(penalizedCost)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs):
            if epoch %10 == 0: #print debugging output
                print("Epoch", epoch, "; penalizedCost =", penalizedCost.eval())
            sess.run(training_op)
        #done training, get final beta: 
        best_beta = beta.eval()
    print(best_beta)
    calc_beta(best_beta, X_test, Y_test)
    return best_beta

In [55]:
#Transforming rdd to tensors for applying Machine Learning Models
X_ghi=GHI_rdd2.map(lambda x: x[1:]).filter(lambda x: x[0]!="GHI1992")
X_ghi=np.array(X_ghi.collect())
print(X_ghi)

best_beta_val = RidgeRegression(X_ghi, 1)

[[ 50.2  52.7  37.9  33.3]
 [ 20.8  21.6  16.5  11.1]
 [ 17.5  15.6  11.3   9.5]
 [ 65.8  57.5  39.7  32.5]
 [  7.    6.6   5.8   5.4]
 [ 23.8  18.4  11.4   7.7]
 [ 36.5  27.5  15.3   9.6]
 [ 53.6  37.6  32.2  26.5]
 [  5.    4.    3.    2. ]
 [ 44.5  37.5  31.7  24.4]
 [ 36.7  30.3  23.9  17.2]
 [ 33.8  33.   30.7  24.4]
 [ 15.9  11.7   5.4   5.4]
 [  7.9   8.2   7.6   5.4]
 [ 47.   47.9  36.4  27.6]
 [ 45.8  43.6  27.1  22.2]
 [ 40.   39.6  29.5  22.1]
 [ 52.2  50.9  47.   50.9]
 [ 62.5  51.9  50.9  43.5]
 [  5.9   4.    3.    2. ]
 [ 25.9  15.8  11.2   7.5]
 [ 14.6  11.3   9.4   8. ]
 [ 39.1  36.   31.6  25.6]
 [  7.5   6.2   5.    5.3]
 [ 32.9  32.6  35.1  26.5]
 [  7.8   6.2   4.    3. ]
 [ 10.5   5.3   4.    3. ]
 [ 60.3  46.7  35.1  31.4]
 [ 23.8  18.4  15.4  11.6]
 [ 22.3  20.5  16.4  14.4]
 [ 20.1  16.4  16.6  14.7]
 [ 19.5  16.2  12.7  11.1]
 [  7.8   6.2   4.    3. ]
 [ 67.8  56.   40.2  32.3]
 [ 11.5   9.8   9.1   8.1]
 [ 24.2  20.7  17.4  13.8]
 [ 35.2  27.5  23.8  23.2]
 

In [56]:
sess = tf.InteractiveSession()
best_beta_val = np.array(best_beta_val, dtype=np.float64)
X_ghi_2024 = tf.matmul(X_ghi[:,1:], best_beta_val, name="predictions")
s = X_ghi_2024.eval()
print(s)

#close the session to release resources
sess.close()

X_ghi_df = pd.DataFrame(X_ghi)
X_ghi_df = X_ghi_df.rename(index=str, columns={0: "GHI1992", 1: "GHI2000",2: "GHI2008",3: "GHI2015"})
X_ghi_df['GHI2024'] = s

[[ 26.4546976 ]
 [ 12.43469454]
 [  8.05606701]
 [ 31.36724661]
 [  2.49178531]
 [ 11.80558237]
 [ 18.92240677]
 [ 16.94490308]
 [  2.3401095 ]
 [ 18.20898169]
 [ 16.37741897]
 [ 14.05774471]
 [  7.25034898]
 [  3.91707274]
 [ 25.67786075]
 [ 25.44863084]
 [ 21.7069727 ]
 [ 13.23394812]
 [ 18.74767412]
 [  2.3401095 ]
 [  9.52140455]
 [  5.07931014]
 [ 16.05158435]
 [  2.21153874]
 [ 12.1916055 ]
 [  3.71377666]
 [  2.87650816]
 [ 22.18318198]
 [  9.1774141 ]
 [  9.30986084]
 [  5.29725149]
 [  7.54448445]
 [  3.71377666]
 [ 30.0818231 ]
 [  3.63060593]
 [  9.8435452 ]
 [  9.94879223]
 [  8.4723056 ]
 [ 16.06218547]
 [ 11.50664034]
 [ 21.51713009]
 [ 19.47430527]
 [  7.31654045]
 [ 16.41262318]
 [  9.44561574]
 [ 14.25809344]
 [  8.69568123]
 [  7.49872361]
 [  9.14315855]
 [  2.44463488]
 [  5.08006651]
 [  6.42999595]
 [ 20.54453752]
 [  2.3401095 ]
 [ 11.92414535]
 [ 26.03302527]
 [  4.17892582]
 [  2.91796221]
 [ 14.51591212]
 [ 20.9574823 ]
 [  3.43468716]
 [  3.55783854]
 [ 14.83

In [58]:
X_ghi_2024 = np.array(X_ghi_df)
sess = tf.InteractiveSession()
best_beta_val = np.array(best_beta_val, dtype=np.float64)
X_ghi_2032 = tf.matmul(X_ghi_2024[:,2:], best_beta_val, name="predictions")
s = X_ghi_2032.eval()
print(s)

#close the session to release resources
sess.close()

X_ghi_2024 = pd.DataFrame(X_ghi_2024)
X_ghi_2024 = X_ghi_2024.rename(index=str, columns={0: "GHI1992", 1: "GHI2000",2: "GHI2008",3: "GHI2015",4: "GHI2024"})
X_ghi_2024['GHI2032'] = s
X_ghi_2024

[[ 17.21427419]
 [  7.0283629 ]
 [  5.04003536]
 [ 15.78326082]
 [  3.61669148]
 [  2.80450049]
 [  1.82625751]
 [ 18.21643366]
 [  1.22796637]
 [ 17.01867095]
 [ 11.1834275 ]
 [ 18.73637887]
 [  0.20916213]
 [  4.38206091]
 [ 16.51445957]
 [  8.19847681]
 [ 12.82144595]
 [ 33.49543235]
 [ 33.86626077]
 [  1.22796637]
 [  4.08250164]
 [  5.22395136]
 [ 18.25968304]
 [  3.05472808]
 [ 23.94634969]
 [  1.24691936]
 [  1.78099926]
 [ 17.40086935]
 [  8.06524797]
 [  8.8127653 ]
 [ 11.54787126]
 [  6.61261438]
 [  1.24691936]
 [ 17.07538174]
 [  5.86545606]
 [  9.42369839]
 [ 14.98048066]
 [  2.04681847]
 [  9.55901864]
 [ 12.58604122]
 [ 16.3425212 ]
 [ 15.71479813]
 [ 10.66702465]
 [ 27.96075661]
 [  9.28785901]
 [ 22.92130302]
 [ 20.00828877]
 [  3.0434804 ]
 [ 17.2724797 ]
 [  5.23003067]
 [  2.57124087]
 [  5.83505053]
 [ 13.69459024]
 [  1.22796637]
 [  4.53329897]
 [ 13.50052149]
 [  0.95020831]
 [  5.48277135]
 [ 16.31496686]
 [ 21.58094536]
 [  1.424946  ]
 [  3.49836309]
 [ 23.42

,GHI1992,GHI2000,GHI2008,GHI2015,GHI2024,GHI2032
0,50.2,52.7,37.9,33.3,26.454698,17.214274
1,20.8,21.6,16.5,11.1,12.434695,7.028363
2,17.5,15.6,11.3,9.5,8.056067,5.040035
3,65.8,57.5,39.7,32.5,31.367247,15.783261
4,7.0,6.6,5.8,5.4,2.491785,3.616691
5,23.8,18.4,11.4,7.7,11.805582,2.804500
6,36.5,27.5,15.3,9.6,18.922407,1.826258
7,53.6,37.6,32.2,26.5,16.944903,18.216434
8,5.0,4.0,3.0,2.0,2.340109,1.227966
9,44.5,37.5,31.7,24.4,18.208982,17.018671


In [59]:
X_ghi_2032 = np.array(X_ghi_2024)
sess = tf.InteractiveSession()
best_beta_val = np.array(best_beta_val, dtype=np.float64)
X_ghi_2040 = tf.matmul(X_ghi_2032[:,3:], best_beta_val, name="predictions")
s = X_ghi_2040.eval()
print(s)

#close the session to release resources
sess.close()

X_ghi_2032 = pd.DataFrame(X_ghi_2032)
X_ghi_2032 = X_ghi_2032.rename(index=str, columns={0: "GHI1992", 1: "GHI2000",2: "GHI2008",3: "GHI2015",4: "GHI2024",5: "GHI2032"})
X_ghi_2032['GHI2040'] = s
X_ghi_2032

[[ 19.06952476]
 [  5.40650744]
 [  5.34006567]
 [ 19.06564867]
 [  2.6291072 ]
 [  4.95991125]
 [  7.10164363]
 [ 12.43808103]
 [  0.99514626]
 [ 11.2041117 ]
 [  8.29246812]
 [ 10.25414511]
 [  4.63566277]
 [  2.09085513]
 [ 14.24049584]
 [ 14.52956164]
 [ 11.61896809]
 [ 25.52141151]
 [ 18.2070964 ]
 [  0.99514626]
 [  4.03882273]
 [  3.93180169]
 [ 11.60458486]
 [  2.90438284]
 [  8.94992808]
 [  1.86513152]
 [  1.55384341]
 [ 17.33288561]
 [  5.32459301]
 [  7.44794963]
 [  6.12322436]
 [  5.84337989]
 [  1.86513152]
 [ 18.10049161]
 [  3.66648364]
 [  6.48133114]
 [ 11.67786004]
 [  5.56024057]
 [  8.4094201 ]
 [ 10.824799  ]
 [ 15.42653537]
 [ 17.75926069]
 [  5.68391555]
 [ 13.40431969]
 [  7.04709953]
 [ 14.08970532]
 [  7.39833648]
 [  4.86563445]
 [  9.96502398]
 [  4.02041538]
 [  4.414508  ]
 [  1.4479187 ]
 [ 10.07948332]
 [  0.99514626]
 [  5.34145406]
 [ 16.05753965]
 [  2.03806936]
 [  3.93560968]
 [ 11.50355058]
 [ 18.33767568]
 [  1.76136882]
 [  2.57413279]
 [ 20.16

,GHI1992,GHI2000,GHI2008,GHI2015,GHI2024,GHI2032,GHI2040
0,50.2,52.7,37.9,33.3,26.454698,17.214274,19.069525
1,20.8,21.6,16.5,11.1,12.434695,7.028363,5.406507
2,17.5,15.6,11.3,9.5,8.056067,5.040035,5.340066
3,65.8,57.5,39.7,32.5,31.367247,15.783261,19.065649
4,7.0,6.6,5.8,5.4,2.491785,3.616691,2.629107
5,23.8,18.4,11.4,7.7,11.805582,2.804500,4.959911
6,36.5,27.5,15.3,9.6,18.922407,1.826258,7.101644
7,53.6,37.6,32.2,26.5,16.944903,18.216434,12.438081
8,5.0,4.0,3.0,2.0,2.340109,1.227966,0.995146
9,44.5,37.5,31.7,24.4,18.208982,17.018671,11.204112


In [109]:
X_param1=GHI_rdd4.map(lambda x: [x[1],x[5],x[9],x[13]]).filter(lambda x: x[0]!="UN9193")
X_param1=np.array(X_param1.collect())
print(X_param1)

best_beta_val_UN1 = RidgeRegression(X_param1, 1)#adjusting penalty: worse

[[  37.4   46.1   24.6   23. ]
 [   7.7    7.2    9.7    4.9]
 [   9.1   10.7    7.5    4.6]
 [  65.4   50.    23.7   14. ]
 [   4.7    3.5    4.1    3.6]
 [  52.7   23.8    5.2    4.4]
 [  81.7   23.2    2.6    1.2]
 [  36.1   20.8   16.4   15.1]
 [   8.2    2.3    1.6    1.1]
 [  28.    22.6   12.8   10.3]
 [  35.9   33.4   28.1   20.2]
 [  26.8   35.6   31.9   26. ]
 [  14.2   12.     2.6    2.3]
 [   1.9    4.8    5.8    3.4]
 [  22.7   25.4   22.6   20.2]
 [  30.9   29.2   18.7   15.3]
 [  36.5   30.6   14.5    7.9]
 [  47.7   42.6   34.7   58.6]
 [  55.7   40.1   40.7   32.5]
 [   7.4    4.7    4.1    3.7]
 [  23.3   15.9   13.5    9.6]
 [  12.6    9.7    9.5    7.1]
 [  43.7   32.5   33.    28.2]
 [   5.5    5.1    5.1    5.6]
 [  13.3   17.6   16.5   15.4]
 [  20.9   10.4    2.2    1.4]
 [  13.2    3.7    2.3    0.8]
 [  75.7   48.1   24.1   12.8]
 [  29.2   28.1   20.2   13.5]
 [  16.2   18.5   14.6   12.1]
 [   5.8    5.3    4.6    4.5]
 [  16.    11.2   11.6   12.3]
 [  13.5

In [110]:
sess = tf.InteractiveSession()
best_beta_val_UN1 = np.array(best_beta_val_UN1, dtype=np.float64)
X_param1_2024 = tf.matmul(X_param1[:,1:], best_beta_val_UN1, name="predictions")
s = X_param1_2024.eval()
print(s)

#close the session to release resources
sess.close()

X_param1_df = pd.DataFrame(X_param1)
X_param1_df = X_param1_df.rename(index=str, columns={0: "UN9193", 1: "UN9901",2: "UN0709",3: "UN1416"})
s[s < 0] = 0
X_param1_df['UN2325'] = s

[[  7.56392551]
 [  2.27872072]
 [  1.62415511]
 [  3.42848881]
 [  1.54843478]
 [  0.1209874 ]
 [ -1.3855872 ]
 [  5.85750847]
 [  0.39357373]
 [  3.37885963]
 [  7.91474874]
 [ 10.45605572]
 [  0.09234589]
 [  1.51000399]
 [  8.12147676]
 [  5.38973964]
 [  1.8238047 ]
 [ 23.45476734]
 [ 13.55242065]
 [  1.47944835]
 [  3.77295051]
 [  2.93281669]
 [ 11.74799132]
 [  2.31790048]
 [  6.2834905 ]
 [ -0.16845512]
 [  0.20920306]
 [  3.14873397]
 [  4.89586037]
 [  4.66541675]
 [  1.80047683]
 [  5.12939692]
 [  0.78023262]
 [ 10.4646397 ]
 [  1.85860588]
 [  2.98632618]
 [  4.35222445]
 [  2.27615264]
 [  2.27271471]
 [  6.02389558]
 [  6.54843686]
 [ 11.4672753 ]
 [  3.80778352]
 [ 19.19133762]
 [  5.76785351]
 [  5.93183441]
 [  3.23658298]
 [  2.41969486]
 [ 11.64337258]
 [  3.56160011]
 [  1.33456067]
 [  0.64338792]
 [  7.33415834]
 [  0.93204964]
 [  2.0693757 ]
 [  5.82723897]
 [  0.23849116]
 [  2.41430643]
 [  5.86967094]
 [ 17.60769182]
 [  0.59426218]
 [  1.26813832]
 [ 17.30

In [111]:
X_param1_df_2024 = np.array(X_param1_df)
sess = tf.InteractiveSession()
best_beta_val_UN1 = np.array(best_beta_val_UN1, dtype=np.float64)
X_param1_2032 = tf.matmul(X_param1_df_2024[:,2:], best_beta_val_UN1, name="predictions")
s = X_param1_2032.eval()
print(s)

#close the session to release resources
sess.close()

X_param1_df_2024 = pd.DataFrame(X_param1_df_2024)
X_param1_df_2024 = X_param1_df_2024.rename(index=str, columns={0: "UN9193", 1: "UN9901",2: "UN0709",3: "UN1416",4:"UN2325"})
s[s < 0] = 0
X_param1_df_2024['UN3133'] = s

[[  3.04988190e+00]
 [  5.14694467e-01]
 [  4.19147228e-01]
 [  5.74277080e-01]
 [  6.04016841e-01]
 [ -5.74843565e-03]
 [ -1.23441732e-01]
 [  2.34517518e+00]
 [  1.20095709e-01]
 [  1.19885092e+00]
 [  2.60482759e+00]
 [  3.85427290e+00]
 [  1.97666763e-02]
 [  4.13260614e-01]
 [  3.19381911e+00]
 [  1.96160583e+00]
 [  1.75328114e-01]
 [  1.20511384e+01]
 [  4.94006933e+00]
 [  5.85294409e-01]
 [  1.22930696e+00]
 [  1.01199204e+00]
 [  4.49403513e+00]
 [  1.01948187e+00]
 [  2.53946904e+00]
 [ -6.76224649e-02]
 [ -4.84279557e-02]
 [  3.07950843e-01]
 [  1.44915653e+00]
 [  1.73393555e+00]
 [  7.47745136e-01]
 [  2.21842219e+00]
 [  3.02749609e-01]
 [  3.78671909e+00]
 [  8.08666062e-01]
 [  9.60195910e-01]
 [  1.79307792e+00]
 [  9.71403309e-01]
 [  1.07328087e+00]
 [  2.50713849e+00]
 [  2.65672033e+00]
 [  5.23184013e+00]
 [  1.33994372e+00]
 [  7.54066575e+00]
 [  2.35301046e+00]
 [  2.32737829e+00]
 [  5.25661666e-01]
 [  9.32489001e-01]
 [  4.72450285e+00]
 [  1.51356713e+00]


In [113]:
X_param1_df_2032 = np.array(X_param1_df_2024)
sess = tf.InteractiveSession()
best_beta_val_UN1 = np.array(best_beta_val_UN1, dtype=np.float64)
X_param1_2040 = tf.matmul(X_param1_df_2032[:,3:], best_beta_val_UN1, name="predictions")
s = X_param1_2040.eval()
print(s)

#close the session to release resources
sess.close()

X_param1_df_2032 = pd.DataFrame(X_param1_df_2032)
X_param1_df_2032 = X_param1_df_2032.rename(index=str, columns={0: "UN9193", 1: "UN9901",2: "UN0709",3: "UN1416",4:"UN2325",5:"UN3133"})
s[s < 0] = 0
X_param1_df_2032['UN3941'] = s

[[ -1.33130946e-01]
 [ -1.98706931e-02]
 [ -9.42389766e-02]
 [ -7.19546445e-01]
 [  6.60142922e-02]
 [ -3.91769499e-01]
 [ -1.10005760e-01]
 [  1.38612412e-01]
 [ -1.38901323e-02]
 [ -1.28918161e-01]
 [ -2.54166297e-02]
 [  1.98741776e-01]
 [ -1.93893782e-01]
 [  2.42177593e-03]
 [  2.35978673e-01]
 [ -8.18499228e-02]
 [ -4.77650662e-01]
 [  1.81722232e+00]
 [  3.44751163e-01]
 [  4.25616438e-02]
 [ -1.45197269e-02]
 [  4.50721389e-02]
 [  3.83197973e-01]
 [  1.26771299e-01]
 [  2.31598245e-01]
 [ -1.28340054e-01]
 [ -5.33052693e-02]
 [ -7.45573407e-01]
 [ -1.74335185e-01]
 [  4.87535600e-02]
 [  6.66002430e-02]
 [  2.73580135e-01]
 [ -3.02828713e-02]
 [ -8.48265075e-02]
 [  8.79885075e-02]
 [  3.81165737e-02]
 [  1.53029899e-01]
 [ -2.52877576e-02]
 [ -3.88303378e-02]
 [  1.75145081e-01]
 [  1.12553377e-01]
 [  6.49793998e-01]
 [  1.35036723e-01]
 [  6.40541961e-01]
 [  1.60743068e-01]
 [  1.93432090e-01]
 [ -1.98667853e-01]
 [  1.10001709e-01]
 [  5.04385280e-01]
 [  1.91850137e-01]


In [117]:
X_param1_df_2032

,UN9193,UN9901,UN0709,UN1416,UN2325,UN3133,UN3941
0,37.4,46.1,24.6,23.0,7.563926,3.049882,0.000000
1,7.7,7.2,9.7,4.9,2.278721,0.514694,0.000000
2,9.1,10.7,7.5,4.6,1.624155,0.419147,0.000000
3,65.4,50.0,23.7,14.0,3.428489,0.574277,0.000000
4,4.7,3.5,4.1,3.6,1.548435,0.604017,0.066014
5,52.7,23.8,5.2,4.4,0.120987,0.000000,0.000000
6,81.7,23.2,2.6,1.2,0.000000,0.000000,0.000000
7,36.1,20.8,16.4,15.1,5.857508,2.345175,0.138612
8,8.2,2.3,1.6,1.1,0.393574,0.120096,0.000000
9,28.0,22.6,12.8,10.3,3.378860,1.198851,0.000000


In [118]:
X_param2=GHI_rdd4.map(lambda x: [x[2],x[6],x[10],x[14]]).filter(lambda x: x[0]!="stu9094")
X_param2=np.array(X_param2.collect())
print(X_param2)

best_beta_val_STU = RidgeRegression(X_param2, 1)#adjusting penalty: worse

[[ 51.3  54.7  51.3  40.9]
 [ 37.2  39.2  23.1  17.9]
 [ 22.9  23.6  13.2  11.7]
 [ 52.9  47.1  29.2  37.6]
 [  7.1   9.9   8.2   7.5]
 [ 23.5  17.7  20.8   9.4]
 [ 30.   24.1  26.8  18. ]
 [ 71.5  54.   43.2  36.1]
 [  7.5   6.    4.2   2.8]
 [ 43.7  39.1  44.7  34. ]
 [ 37.1  33.1  27.2  18.1]
 [ 41.   29.1  31.4  23.5]
 [ 13.3  10.    7.1   7.9]
 [ 12.8  10.2   8.1   6.2]
 [ 40.7  45.5  35.1  27.3]
 [ 58.5  49.2  39.5  32.4]
 [ 36.3  38.2  36.4  31.7]
 [ 42.6  44.6  45.1  42.6]
 [ 47.2  39.3  38.7  39.9]
 [  4.2   3.    2.    1.8]
 [ 38.   17.8   9.8   6.3]
 [ 22.   18.1  12.7  12.2]
 [ 32.2  28.9  28.8  21.2]
 [ 10.7   7.9   5.6   5.2]
 [ 34.2  31.5  39.   29.6]
 [  1.3   1.4   1.3   1. ]
 [  8.6   7.    6.1   4.4]
 [ 34.3  26.5  33.   33.5]
 [ 21.2   8.   10.1   7.1]
 [ 38.1  32.5  26.3  25.2]
 [ 33.1  24.6  30.7  22.3]
 [ 29.5  32.3  20.6  13.6]
 [ 11.4   5.4   4.4   3.8]
 [ 66.9  57.4  46.4  38.4]
 [  4.3   5.8   6.4   4.9]
 [ 30.3  26.3  19.2  17.5]
 [ 33.9  24.1  25.5  25. ]
 

In [ ]:
['stu9094' 'stu9802' 'stu0610' 'stu1216']

In [27]:
X_param3=GHI_rdd4.map(lambda x: [x[3],x[7],x[11],x[15]])
X_param3=np.array(X_param3.collect())
print(X_param3)


RidgeRegression(X_param3, 1)
RidgeRegression(X_param3, .1)#adjusting penalty: worse
RidgeRegression(X_param3, 10)#better
RidgeRegression(X_param3, 100)#worse
RidgeRegression(X_param3, 10, .01) #gives error because learning rate is too high
RidgeRegression(X_param3, 10, .00001, 10000) #lower learning rate, more epochs: finds finer-grained solution

[['wast9094' 'wast9802' 'wast0610' 'wast1216']
 ['11.5' '13.4' '8.9' '9.5']
 ['9.4' '12.2' '9.4' '6.2']
 ['7.1' '3.1' '4.1' '4.1']
 ['7.9' '8.7' '8.2' '4.9']
 ['1.6' '1.6' '1.2' '1.6']
 ['5.0' '2.5' '4.2' '4.2']
 ['6.6' '9.0' '6.8' '3.1']
 ['16.1' '13.8' '17.5' '14.3']
 ['2.6' '2.0' '1.9' '1.8']
 ['11.2' '9.0' '8.4' '4.5']
 ['3.6' '1.6' '1.4' '1.6']
 ['13.1' '6.0' '7.2' '6.8']
 ['2.9' '2.2' '1.6' '1.9']
 ['3.4' '3.2' '3.4' '2.8']
 ['15.5' '15.7' '11.3' '7.6']
 ['13.9' '16.9' '8.9' '9.6']
 ['4.5' '6.2' '7.3' '5.2']
 ['9.7' '10.5' '12.2' '7.2']
 ['14.6' '13.9' '15.7' '13.0']
 ['0.5' '0.5' '0.3' '0.3']
 ['3.9' '2.5' '2.6' '1.8']
 ['1.7' '1.1' '0.9' '1.1']
 ['7.1' '7.1' '7.2' '8.2']
 ['2.1' '1.7' '1.0' '1.4']
 ['8.3' '6.9' '14.0' '7.6']
 ['1.6' '1.3' '1.0' '1.2']
 ['3.3' '2.4' '2.4' '2.0']
 ['17.3' '19.4' '17.0' '21.5']
 ['2.2' '1.5' '2.3' '2.4']
 ['2.8' '3.2' '2.7' '2.3']
 ['4.3' '6.9' '7.9' '9.5']
 ['1.4' '1.5' '1.6' '2.1']
 ['4.6' '2.7' '2.5' '2.5']
 ['9.2' '12.4' '10.8' '9.9']
 ['9.8' 

ValueError: could not convert string to float: 'wast9094'

In [28]:
X_param4=GHI_rdd4.map(lambda x: [x[4],x[8],x[12],x[16]])
X_param4=np.array(X_param4.collect())
print(X_param4)



RidgeRegression(X_param4, 1)
RidgeRegression(X_param4, .1)#adjusting penalty: worse
RidgeRegression(X_param4, 10)#better
RidgeRegression(X_param4, 100)#worse
RidgeRegression(X_param4, 10, .01) #gives error because learning rate is too high
RidgeRegression(X_param4, 10, .00001, 10000) #lower learning rate, more epochs: finds finer-grained solution

[['UM1992' 'UM2000' 'UM2008' 'UM2015']
 ['16.8' '13.7' '11.0' '9.1']
 ['3.7' '2.6' '1.8' '1.4']
 ['4.5' '4.0' '2.9' '2.6']
 ['22.6' '21.7' '19.2' '15.7']
 ['2.6' '2.0' '1.5' '1.3']
 ['4.5' '3.0' '2.0' '1.4']
 ['9.5' '7.4' '4.3' '3.2']
 ['13.2' '8.8' '5.6' '3.8']
 ['1.7' '1.4' '0.7' '0.5']
 ['17.0' '14.5' '11.6' '10.0']
 ['11.5' '8.0' '5.2' '3.8']
 ['5.9' '8.3' '6.2' '4.4']
 ['5.5' '3.2' '1.8' '1.6']
 ['2.3' '2.1' '1.4' '1.0']
 ['20.2' '18.6' '13.0' '8.9']
 ['11.8' '10.8' '5.2' '2.9']
 ['14.3' '15.0' '11.3' '8.8']
 ['17.6' '17.5' '15.8' '13.0']
 ['20.9' '19.0' '16.8' '13.9']
 ['1.6' '1.1' '0.9' '0.8']
 ['5.2' '3.7' '1.9' '1.1']
 ['3.3' '2.5' '2.0' '1.6']
 ['9.7' '12.2' '7.3' '4.5']
 ['1.6' '1.3' '1.0' '1.0']
 ['15.3' '14.6' '11.7' '9.3']
 ['1.1' '0.8' '0.6' '0.4']
 ['1.2' '0.8' '0.6' '0.6']
 ['11.5' '10.1' '8.1' '6.5']
 ['5.6' '4.1' '3.5' '3.1']
 ['5.2' '3.4' '2.7' '2.2']
 ['7.8' '4.7' '3.1' '2.4']
 ['5.3' '3.2' '2.2' '1.7']
 ['1.9' '1.1' '0.6' '0.3']
 ['19.5' '14.5' '8.7' '5.9']
 ['2.8

ValueError: could not convert string to float: 'UM1992'